# Quadratic Assignment Problem (QAP)

In this project, we address the **Quadratic Assignment Problem (QAP)**, a classic and computationally challenging combinatorial optimization problem. QAP arises in various real-world scenarios, such as facility layout design, scheduling, and electronics design, where a set of facilities must be assigned to a set of locations in a way that minimizes the total cost, typically based on the distance and flow between the facilities.

We aim to solve QAP instances by implementing and evaluating a variety of **Local Search** and **Population-Based Search** algorithms. The solution process begins with algorithm development and is followed by experimental analysis using both small-scale and larger, more complex problem instances.

## Algorithms Implemented

The following families of metaheuristic algorithms will be implemented:

### Local Search Methods

These algorithms iteratively explore the solution space by making small, localized changes to the current solution.

- **First Improvement**  
  Performs a neighborhood search and accepts the first neighboring solution that improves the current objective value.

- **Best Improvement**  
  Explores all neighboring solutions and selects the one with the best improvement (or least deterioration, in case of local optima escape).

- **Tabu Search**  
  Enhances local search by using memory structures that describe the visited solutions or user-defined attributes, in order to avoid cycles and encourage exploration of new areas.

### Population-Based Search Methods

These methods maintain and evolve a population of candidate solutions over time, typically offering greater exploration capabilities than local search.

- **Ant Colony Optimization (ACO)**  
  A probabilistic technique inspired by the foraging behavior of ants. It uses pheromone trails to guide the search towards promising regions of the solution space.

- **(μ + λ) Evolutionary Algorithm**  
  A type of evolutionary strategy where μ parents generate λ offspring through variation operators (mutation, crossover), and the next generation is selected from the combined pool of parents and offspring.

## Initial Setup

To begin, we load all the required libraries and modules necessary for the implementation of these algorithms. This includes standard libraries for numerical computation, data handling, and optimization, as well as custom modules for QAP-specific operations.


In [3]:
import numpy as np
import random

## Problem Instance: 9-City Case

To begin our experimentation, we will use a simplified instance of the Quadratic Assignment Problem involving **9 cities**. This small-scale case provides a manageable context for developing, debugging, and evaluating the performance of our algorithms before scaling to larger instances.

### Data Description

We define two key matrices that model the problem:

- **Cost Matrix (`qap_costs_9`)**  
  This matrix represents the cost associated with traveling from one city to another. Each entry `C[i][j]` indicates the cost incurred when assigning facility `i` to location `j`. These costs may encapsulate factors such as resource usage, traffic load, or economic penalties.

- **Distance Matrix (`qap_distances_9`)**  
  This matrix encodes the physical or logical distances between locations. Each element `D[i][j]` specifies the distance between location `i` and location `j`. In real-world analogs, this could reflect geographic distances, travel times, or network latencies.

### Objective

The goal of the QAP is to determine an optimal assignment of facilities (cities) to locations that **minimizes the total cost**, which is calculated as the sum of the products of corresponding entries in the cost and distance matrices. Formally, the objective function is:

$$
\text{Minimize} \quad \sum_{i=1}^{n} \sum_{j=1}^{n} C_{i,j} \cdot D_{\pi(i), \pi(j)}
$$

Where:

- $C_{i,j}$ is the flow (or cost) between facility $i$ and facility $j$.
- $D_{\pi(i), \pi(j)}$ is the distance between the locations assigned to facilities $i$ and $j$.
- $\pi$ is a permutation of the set $\{1, 2, \ldots, n\}$ representing an assignment of facilities to locations.

This instance requires finding a permutation of the 9 cities such that the total weighted cost, combining both the flow and the distance, is minimized while visiting all cities exactly once.

In [4]:
qap_cost_9 =  [
    [0, 21, 95, 82, 56, 41, 6, 25, 10], 
    [21, 0, 4, 63, 6, 44, 40, 75, 79],
    [95, 4, 0, 0, 89, 35, 9, 1, 85], 
    [82, 63, 0, 0, 84, 12, 0, 26, 91], 
    [56, 6, 89, 84, 0, 11, 35, 82, 26], 
    [41, 44, 35, 12, 11, 0, 69, 56, 86], 
    [6, 40, 9, 0, 35, 69, 0, 45, 91], 
    [25, 75, 1, 26, 82, 56, 45, 0, 59], 
    [10, 79, 85, 91, 26, 86, 91, 59, 0]
]

qap_distances_9 = [
    [0, 18, 76, 39, 18, 57, 36, 61, 36], 
    [18, 0, 21, 71, 11, 29, 82, 82, 6], 
    [76, 21, 0, 71, 8, 77, 74, 30, 89], 
    [39, 71, 71, 0, 76, 76, 40, 93, 56], 
    [18, 11, 8, 76, 0, 1, 50, 4, 36], 
    [57, 29, 77, 76, 1, 0, 27, 85, 2], 
    [36, 82, 74, 40, 50, 27, 0, 1, 15], 
    [61, 82, 30, 93, 4, 8, 1, 0, 11], 
    [36, 6, 89, 56, 36, 2, 15, 11, 0]
]

# Best solution: 94622

## Loading the 35-City QAP Data

For the larger problem instance involving **35 cities**, we load the corresponding **cost** and **distance** matrices from text files. These files contain space-separated integers, where each line corresponds to a row of the matrix.

To facilitate the loading process, we define a concise function that reads and parses the data into a 2D list (i.e., a matrix) of integers.

In [ ]:
# Load cost and distance matrices for the 35-city problem instance
with open('qap_cost_35.txt') as f:
    qap_cost_35 = [[int(num) for num in line.split()] for line in f]

with open('qap_dist_35.txt') as f:
    qap_distances_35 = [[int(num) for num in line.split()] for line in f]
    
# Best solution: 2422002

### Common Utility Functions

Before implementing specific algorithms, we begin by defining several **utility functions** that are shared across all methods. These common functions help modularize the code and promote reusability and clarity.

#### Generating a Random Initial Solution

One of the fundamental operations in metaheuristic algorithms is the generation of an initial candidate solution. The function `randomSolution` accomplishes this by creating a random permutation of city indices, representing a valid solution to the QAP.

This random starting point is especially useful for stochastic algorithms like Local Search or Evolutionary Strategies, where diversity in starting conditions can improve solution quality.


In [ ]:
def randomSolution(qap_cost, seed=None):
    """
    Generates a random solution (permutation) for the QAP problem.

    Parameters:
    - qap_cost: 2D list (cost matrix), used to determine the number of cities.
    - seed: Optional integer to set the random seed for reproducibility.

    Returns:
    - A list representing a random permutation of city indices.
    """
    if seed is not None:
        random.seed(seed)
    solution = list(range(len(qap_cost[0])))
    random.shuffle(solution)
    return solution

#### Evaluating a Solution: `calculateCost`

Once a solution (i.e., a permutation of cities or facilities) is generated, we need a way to **evaluate its quality**. The `calculateCost` function computes the **total cost** associated with a specific assignment by summing the weighted interactions between all pairs of facilities.

#### Description

The objective function of the Quadratic Assignment Problem is based on two components:

- **Flow matrix (`qap_cost`)**: Represents the flow or cost between facility pairs.
- **Distance matrix (`qap_distances`)**: Represents the distance between location pairs.

The function multiplies the flow between each pair of facilities by the distance between the locations assigned to them and sums over all pairs.

In [ ]:
def calculateCost(qap_cost, qap_distances, solution):
    """
    Computes the total cost of a given solution to the QAP.

    Parameters:
    - qap_cost: 2D list, flow/cost matrix between facilities.
    - qap_distances: 2D list, distance matrix between locations.
    - solution: List[int], a permutation representing facility-to-location assignments.

    Returns:
    - Integer representing the total cost of the solution.
    """
    cost = 0
    n = len(solution)
    for i in range(n):
        for j in range(n):
            cost += qap_cost[i][j] * qap_distances[solution[i]][solution[j]]
    return cost

#### Generating a Neighboring Solution: `two_opt_swap`

One common operation in optimization algorithms, particularly in local search methods, is to explore neighboring solutions by making small changes to the current solution. The **2-opt swap** is a widely used technique in routing problems, where the goal is to improve the current solution by reversing a portion of the path. This method is especially effective in problems like the **Traveling Salesman Problem (TSP)** or **Quadratic Assignment Problem (QAP)**.

#### Description

The `two_opt_swap` function takes a given route (solution) and reverses the order of cities (nodes) between two indices, `i` and `k`. This operation generates a new solution that can potentially lead to a better cost when evaluated by the objective function.

In [ ]:
def two_opt_swap(route, i, k):
    """
    Perform a 2-opt swap on the given route by reversing the
    order of the nodes between indices i and k (inclusive).
    
    Parameters:
    - route: List[int], a permutation of city indices representing the current solution.
    - i: Integer, the starting index of the segment to reverse.
    - k: Integer, the ending index of the segment to reverse.
    
    Returns:
    - new_route: List[int], a new solution where the order of cities between i and k is reversed.
    """
    new_route = route.copy()
    new_route[i:k+1] = reversed(route[i:k+1])
    
    return new_route

#### Generating Neighbors: `get_neighbors`

In many optimization algorithms, it's essential to explore the **neighborhood** of a given solution to identify potential improvements. The `get_neighbors` function helps generate all neighboring solutions by applying specific transformations to the current solution. In this case, the transformation is a **2-opt swap**, which reverses segments of the solution.

#### Description

The `get_neighbors` function generates a list of all possible neighbors for a given path (solution). It iterates over all pairs of indices \(i, j\) in the path, and for each pair, it creates a new solution by reversing the portion of the path between indices \(i\) and \(j\). This forms the neighborhood of the current solution.


In [ ]:
def get_neighbors(path):
    """
    Generates the neighbors of a given solution by performing all possible 2-opt swaps.
    
    Parameters:
    - path: List[int], the current solution (a permutation of city indices).
    
    Returns:
    - neighbors: List[List[int]], a list of neighbor solutions generated by 2-opt swaps.
    """
    neighbors = []
    for i in range(len(path)):
        for j in range(i+1, len(path)):
            neighbor = np.copy(path)
            neighbor[i:j+1] = np.flip(neighbor[i:j+1])
            neighbors.append(neighbor)
    return neighbors

#### Selecting the Best Neighbor: `getBestNeighbour`

In many local search algorithms, once a set of neighbors is generated, the next step is to evaluate them and select the best one. The `getBestNeighbour` function does exactly that: it evaluates all neighbors based on their cost and returns the one with the lowest cost.

#### Description

The function `getBestNeighbour` takes a list of neighbors and calculates the cost of each solution using the `calculateCost` function. It then returns the neighbor with the **minimum cost**. This is particularly useful for algorithms like **First Improvement** or **Best Improvement**, where the goal is to select the most promising neighbor to move to in the solution space.

In [ ]:
def getBestNeighbour(qap_cost, qap_distances, neighbors):
    """
    Selects the best neighbor from a list of neighbors by evaluating their cost.
    
    Parameters:
    - qap_cost: 2D list (cost matrix), used to compute the total cost of the solution.
    - qap_distances: 2D list (distance matrix), used to compute the total cost of the solution.
    - neighbors: List[List[int]], a list of neighbor solutions (permutations of cities).
    
    Returns:
    - Tuple containing:
      - The best neighbor (List[int]): The solution with the minimum cost.
      - The cost (float): The cost of the best neighbor.
    """
    return min(((nb_solution, calculateCost(qap_cost, qap_distances, nb_solution)) for nb_solution in neighbors), key=lambda x: x[1])

We have finished defining all the common function, so now we begin with the first algorithm, the `first improvement`.

## First Improvement

The **First Improvement** algorithm is a local search heuristic designed to explore the solution space efficiently by making incremental improvements to the current solution. Unlike exhaustive search methods that evaluate all possible solutions, First Improvement stops as soon as it finds the first solution that improves the current one. 

### Key Features
- **Efficiency**: First Improvement is computationally more efficient than exhaustive search algorithms because it does not require evaluating every possible neighbor. As soon as a better solution is found, the search proceeds to that neighbor.
- **Local Optima**: While this method is fast, it has the potential downside of getting stuck in **local optima**. Since the algorithm only considers the first improving neighbor, it may miss opportunities for a global optimum by not exploring the entire neighborhood.

### Algorithm Overview
1. **Initialization**: The algorithm begins with an initial solution, which could be a random solution or a starting heuristic.
2. **Neighbor Exploration**: All possible neighbors of the current solution are generated.
3. **Improvement Check**: The algorithm evaluates each neighbor and accepts the first one that results in an improvement (i.e., a lower cost).
4. **Termination**: The process stops when no improving neighbor can be found, indicating that a local optimum has been reached.

### Advantages
- **Speed**: Since it halts as soon as a better solution is found, it is relatively fast compared to exhaustive search methods.
- **Simplicity**: The algorithm is straightforward and easy to implement, making it a good starting point for more complex optimization techniques.

### Disadvantages
- **Local Optima**: The First Improvement algorithm may stop prematurely, leading to a solution that is only a local optimum and not the best possible global solution.
- **Limited Exploration**: By focusing only on the first improvement found, the algorithm may overlook better solutions that require more exploration.

### Example Scenario
Consider a scenario where we are solving the **Quadratic Assignment Problem (QAP)**. If our current solution is `[0, 1, 2, 3, 4]` and we find a neighbor `[0, 2, 1, 3, 4]` that improves the total cost, the algorithm will accept this neighbor and stop searching further in that iteration. 

This strategy makes First Improvement a useful algorithm when **time efficiency** is more important than finding the absolute best solution. It can be particularly beneficial in **large-scale optimization problems**, where exhaustive search methods are not feasible due to time and computational limitations.

### Use Case

The First Improvement algorithm is widely used in optimization problems where finding an exact global optimum is not strictly necessary. It's commonly applied in:
- **Quadratic Assignment Problem (QAP)**
- **Traveling Salesman Problem (TSP)**
- **Vehicle Routing Problem (VRP)**
- **Job Shop Scheduling**

In [ ]:
def firstImprovement(qap_cost, qap_distances):
    """
    Perform First Improvement local search to find a better solution for QAP.

    Parameters:
    - qap_cost: 2D list (cost matrix), representing the flow between facilities.
    - qap_distances: 2D list (distance matrix), representing the distances between locations.

    Returns:
    - currentSolution: List[int], the best solution found.
    - currentCost: float, the cost of the best solution.
    """
    # Step 1: Generate an initial random solution
    currentSolution = randomSolution(qap_cost)
    currentCost = calculateCost(qap_cost, qap_distances, currentSolution)

    while True:
        bestNeighbour = None
        bestNeighbourCost = float("inf")
        foundBetterNeighbor = False

        # Step 2: Explore the neighbors of the current solution
        for i in range(len(currentSolution)):
            for j in range(i + 1, len(currentSolution)):
                # Generate a new neighbor by performing a 2-opt swap
                neighbour = two_opt_swap(currentSolution, i, j)
                neighbourCost = calculateCost(qap_cost, qap_distances, neighbour)

                # Step 3: Check if the neighbor improves the current solution
                if neighbourCost < currentCost:
                    bestNeighbour = neighbour
                    bestNeighbourCost = neighbourCost
                    foundBetterNeighbor = True
                    break

            if foundBetterNeighbor:
                break

        # Step 4: If no improvement, terminate the search
        if bestNeighbour is None:
            break

        # Step 5: Move to the best improving neighbor and continue the search
        currentSolution = bestNeighbour
        currentCost = bestNeighbourCost

    return currentSolution, currentCost

## First Improvement with Multiple Iterations

In order to obtain a high-quality solution, it is common to run the **First Improvement** algorithm multiple times. This is particularly useful because the algorithm's performance is highly sensitive to the initial solution, which is generated randomly. By executing the algorithm several times with different starting solutions, we increase the chances of finding the global optimum, or at least a much better local optimum.

### Approach

To improve the chances of finding a good solution, we will run the **First Improvement** algorithm for **1000 iterations**. Each time the algorithm is run, it starts with a new random solution. By comparing the best solution found across all iterations, we can ensure that the solution is closer to the optimal one.

In [ ]:
Cost = 100000000  # Initialize with a very large value to ensure we can improve upon it

for i in range(1000):
    # Run the First Improvement algorithm to find a better solution in each iteration
    currentSolution, currentCost = firstImprovement(qap_cost_9, qap_distances_9)

    # If the current solution is better, update the best solution found so far
    if currentCost < Cost:
        Solution, Cost = currentSolution, currentCost
Solution, Cost

# Best solution founded: 94622
# Best solution: 94622

([8, 2, 5, 7, 6, 1, 0, 3, 4], 94622)

## First Improvement with 20 Iterations for the 35-Cities Problem

To evaluate the **First Improvement** algorithm for a larger problem instance, we apply it to the **35-cities problem**. Since this problem is significantly larger than the 9-cities case, running the algorithm for **1000 iterations** would be computationally expensive and time-consuming. Therefore, we reduce the number of iterations to **20**, which provides a good balance between solution quality and computational cost.

### Approach

For the 35-cities case, we use the same structure as before but reduce the number of iterations to 20 to avoid excessive computation. This approach allows us to quickly assess the performance of the algorithm while keeping the execution time manageable.

In [ ]:
Cost = 100000000  # Initialize with a very large value to ensure improvement

for i in range(20):
    # Run the First Improvement algorithm for the 35-cities problem
    currentSolution, currentCost = firstImprovement(qap_cost_35, qap_distances_35)

    # If the current solution is better, update the best solution found so far
    if currentCost < Cost:
        Solution, Cost = currentSolution, currentCost
        
np.array(Solution).T, Cost

# Best solution founded: 2593408
# Best solution: 2422002

(array([ 7, 18, 34, 25, 33, 21, 11,  4, 30, 14,  6, 32,  1,  5, 23, 31, 22,
        13, 28, 19, 16,  8, 17,  0, 10, 29,  9,  3, 24, 15, 12, 27,  2, 26,
        20]),
 2593408)

The low computational time, allows us to use the algorithm with a loop, and reach a great solution very fast.

## Best Improvement

**Best Improvement** is a technique that enhances local search in combinatorial optimization problems. Unlike **First Improvement**, which stops the search as soon as an improvement is found, **Best Improvement** explores the solution space more thoroughly. In each iteration, it searches for the **best possible improvement** among all available neighbors, which allows it to find solutions closer to the global optimum.

This algorithm has a higher potential for finding a better solution compared to **First Improvement**, but it also requires more computational time due to the need to explore all possible neighbors before making an update.

### How It Works

1. **Initialization**:
   - A random initial solution is generated.
   - The cost of the solution is calculated.
   - All neighbors of the current solution are generated.

2. **Searching for the Best Improvement**:
   - In each iteration, the algorithm explores all possible neighbors of the current solution.
   - The cost of each neighbor is calculated, and the neighbor with the lowest cost is selected.

3. **Iteration**:
   - If the found neighbor has a lower cost than the current solution, the solution is updated with this better neighbor.
   - The process repeats until no further improvements can be made.

In [ ]:
def bestImprovement(qap_cost, qap_distances):
    """
    Performs the Best Improvement local search algorithm to find an optimal solution for the QAP.

    Parameters:
    - qap_cost: 2D list (cost matrix), representing the flow between facilities.
    - qap_distances: 2D list (distance matrix), representing the distances between locations.

    Returns:
    - currentSolution: List[int], the best solution found.
    - currentCost: float, the cost of the best solution.
    """
    currentSolution = randomSolution(qap_cost)
    currentCost = calculateCost(qap_cost, qap_distances, currentSolution)
    neighbors = get_neighbors(currentSolution)
    bestNeighbour, bestNeighbourCost = getBestNeighbour(qap_cost, qap_distances, neighbors)

    while bestNeighbourCost < currentCost:
        currentSolution = bestNeighbour
        currentCost = bestNeighbourCost
        neighbors = get_neighbors(currentSolution)
        bestNeighbour, bestNeighbourCost = getBestNeighbour(qap_cost, qap_distances, neighbors)

    return currentSolution, currentCost

As in the previous case, we use a loop to fin out the best possible solution, because with one iteration is difficult to reach the best one (it can get stuck in a local optimum)

In [16]:
Cost = 100000000
for i in range(1000):
    currentSolution, currentCost = bestImprovement(qap_cost_9, qap_distances_9)    
    if currentCost < Cost:
        Solution, Cost = currentSolution, currentCost
Solution, Cost

# Best solution founded: 94622
# Best solution: 94622

(array([8, 2, 5, 7, 6, 1, 0, 3, 4]), 94622)

We use a smaller loop, and we can find out that this algorithm is slower than the first improvement one, but this has better results.

In [63]:
Cost = 100000000
for i in range(20):
    currentSolution, currentCost = bestImprovement(qap_cost_35, qap_distances_35)    
    if currentCost < Cost:
        Solution, Cost = currentSolution, currentCost
np.array(Solution).T, Cost 

# Best solution founded: 2503074
# Best solution: 2422002

(array([ 3, 27, 11, 18, 22,  6, 29,  4, 24,  1, 25, 21, 12, 17,  9, 33,  0,
         5, 13, 28,  2, 26, 34, 15, 32,  7, 16, 30, 20, 14, 19, 23, 10, 31,
         8]),
 2614798)

Here, we have also an algorithm with low computational execution time, so we can use a loop to reach a best solution.

## Tabu Search

**Tabu Search** is an advanced optimization algorithm that builds on the concept of exploring the solution space iteratively while maintaining a list of prohibited moves, known as the "tabu list". This mechanism helps avoid revisiting previously explored solutions (thus preventing loops) and encourages diversity in the search process, which ultimately leads to more effective exploration of the solution space.

### How It Works

1. **Initialization**:
   - The algorithm starts with an initial solution, which can be generated randomly or using another method.
   - A tabu list is initialized to store recently visited solutions, and a best solution is tracked throughout the search.

2. **Exploration of Neighbors**:
   - In each iteration, the algorithm generates a set of neighboring solutions.
   - It examines each neighbor and checks if it is in the tabu list (i.e., if it has been visited before). If a neighbor is not in the tabu list, it is considered for further evaluation.
   
3. **Selection of the Best Neighbor**:
   - Among all valid neighbors (those not in the tabu list), the best one is selected based on the cost (the one with the lowest cost).
   
4. **Tabu List Management**:
   - The selected neighbor is then added to the tabu list to prevent revisiting it in future iterations.
   - The size of the tabu list is managed by removing the oldest solutions once the list exceeds the specified size.
   
5. **Updating the Best Solution**:
   - If the newly selected neighbor is better than the current best solution (i.e., it has a lower cost), it is stored as the new best solution.
   
6. **Termination**:
   - The process repeats for a specified number of iterations, or until a stopping condition (such as a set number of iterations) is met.

In [ ]:
def tabu_search(initial_solution, qap_cost, qap_distances, tabu_list_size, max_iterations):
    """
        Perform Tabu Search to optimize the Quadratic Assignment Problem (QAP).

        Parameters:
        - initial_solution: List[int], the starting solution (a permutation of facilities).
        - qap_cost: 2D list, the flow/cost matrix between facilities.
        - qap_distances: 2D list, the distance matrix between locations.
        - tabu_list_size: int, the maximum size of the tabu list.
        - max_iterations: int, the maximum number of iterations to perform.

        Returns:
        - best_solution: List[int], the best solution found during the search.
        - currentCost: float, the cost of the best solution.
    """
    current_solution = np.copy(initial_solution)
    best_solution = np.copy(initial_solution)
    tabu_list = []
    
    for i in range(max_iterations):
        neighbors = get_neighbors(current_solution)
        best_neighbor = None
        
        for neighbor in neighbors:
            if neighbor.tolist() not in tabu_list:
                if best_neighbor is None or calculateCost(qap_cost, qap_distances, neighbor) < calculateCost(qap_cost, qap_distances, best_neighbor):
                    best_neighbor = neighbor
        
        current_solution = best_neighbor
        tabu_list.append(current_solution.tolist())
        
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)
        
        if calculateCost(qap_cost, qap_distances, current_solution) < calculateCost(qap_cost, qap_distances, best_solution):
            best_solution = current_solution
            
    currentCost = calculateCost(qap_cost, qap_distances, best_solution)
    return best_solution, currentCost


### Tabu Search: 9-Cities Problem

We begin by applying the **Tabu Search** algorithm to the 9-cities problem, without the use of a loop. Despite this, the algorithm achieves good results with a tour length of **94622**, and it does so in a relatively fast manner. This showcases the efficiency of Tabu Search even in smaller problem instances.

#### Algorithm Parameters

We set the following parameters for the algorithm:

- **Tabu List Size**: 5 (limits the size of the tabu list to store the most recent solutions).
- **Max Iterations**: 1000 (maximum number of iterations for the search process).

In [ ]:
tabu_list_size = 5
max_iterations = 1000

# Generate a random initial solution
initial_solution = randomSolution(qap_cost_9)

# Apply the Tabu Search algorithm
bestSolution, bestCost = tabu_search(initial_solution, qap_cost_9, qap_distances_9, tabu_list_size, max_iterations)

# Output the best solution and its cost
print("Best route found: ", bestSolution)
print("Best tour length: ", bestCost)

Best route found:  [8 2 5 7 6 1 0 3 4]
Best tour length:  94622


With the 35-cities case, we increase the tabu list size, so as it can find out new solutions. Although its hight computational execution time, it doesn,t reach the best solution.

In [61]:
tabu_list_size = 6
max_iterations = 500

initial_solution = randomSolution(qap_cost_35)
bestSolution, bestCost =  tabu_search(initial_solution, qap_cost_35, qap_distances_35, tabu_list_size, max_iterations)   

print("Best route found: ", bestSolution)
print("Best tour length: ", bestCost)

# Best solution founded: 2608596
# Best solution: 2422002

Best route found:  [22 12  9 17  7 10  4 15  8 23  2 14 11 16 33 20 19 31 28 32  3  6  1 25
 13 21 26 34  5  0 30 18 27 24 29]
Best tour length:  2629144


## Population Search

Population search techniques, such as the **Ant Colony Algorithm** and the **Genetic Algorithm**, are widely used for solving combinatorial optimization problems like the **Quadratic Assignment Problem (QAP)**. These methods utilize a population of candidate solutions and evolve them over time to find better solutions. There are many variants and extensions of these algorithms that can be applied to QAP, depending on the specific problem requirements and constraints.

In this implementation, we will focus on one of the population-based approaches, specifically the **Ant Colony Optimization (ACO)** algorithm, which is based on the behavior of ants exploring the solution space.

### ANTS (Ant Colony Optimization)

In the context of population search for QAP, the **Ant Colony Algorithm** leverages the behavior of ants, which are able to explore the solution space, discover promising solutions, and progressively improve the quality of the solutions through interaction with their environment. Ants communicate via pheromones, which are deposited on the paths they take. Over time, paths with more pheromones become more attractive to other ants, leading to the discovery of better solutions.

The first step in implementing ACO for QAP is to create an **Ant** class. This class will include methods that allow each ant to simulate its behavior, such as exploring possible solutions, updating pheromones, and constructing solutions.

### Ant Class Implementation

The **Ant** class will include the following methods:

1. **Initialization**: Each ant needs to be initialized with a set of parameters, such as its starting position, pheromone levels, and other necessary attributes.
2. **Exploration**: Ants will explore the solution space by moving through different possible assignments of cities, evaluating the cost at each step.
3. **Pheromone Update**: After completing its path, the ant will deposit pheromones on the cities it visited to guide future ants.
4. **Solution Construction**: The ant constructs a solution based on the state of the pheromone trail and possibly other local search mechanisms.

In [ ]:
class Ant:
    """
    Ant Class

    This class represents an individual ant in the Ant Colony Optimization algorithm. Each ant is responsible for constructing a solution to the Quadratic Assignment Problem (QAP) by exploring the solution space and depositing pheromones based on the quality of the solution.

    Methods:
    - __init__: Initializes the ant with the QAP cost and distance matrices.
    - _pick_next_facility: Selects the next facility to visit based on pheromone levels and heuristic information.
    - find_solution: Constructs a complete solution by visiting all facilities and calculates the total cost of the solution.
    """
    def __init__(self, qap_cost, qap_distances):
        self.qap_cost = qap_cost
        self.qap_distances = qap_distances
        self.facilities = []
        self.tour_cost = 0.0

    def _pick_next_facility(self, remaining_facilities, pheromone_matrix):
        current_facility = self.facilities[-1]
        probabilities = []

        for facility in remaining_facilities:
            pheromone = pheromone_matrix[current_facility][facility]
            distance = self.qap_distances[current_facility][facility]
            cost = self.qap_cost[current_facility][facility]
            if distance == 0 or cost == 0:
                probabilities.append(0)
            else:
                probabilities.append(pheromone * (1.0 / (distance * cost)))

        probabilities_sum = sum(probabilities)
        
        if probabilities_sum == 0:
            probabilities = [1 / len(remaining_facilities)] * len(remaining_facilities)
        else:
            probabilities = [prob / probabilities_sum for prob in probabilities]
            
        next_facility_index = np.random.randint(len(remaining_facilities))
        next_facility = remaining_facilities[next_facility_index]
        return next_facility

    def find_solution(self, start_facility, pheromone_matrix):
        self.facilities = [start_facility]
        remaining_facilities = [i for i in range(len(self.qap_cost)) if i != start_facility]

        while remaining_facilities:
            next_facility = self._pick_next_facility(remaining_facilities, pheromone_matrix)
            self.facilities.append(next_facility)
            remaining_facilities.remove(next_facility)

        self.tour_cost = calculateCost(self.qap_cost, self.qap_distances, self.facilities)

Once we have created the class and its methods, we define a function to run this algorithm.

In [ ]:
def ant_colony_optimization(qap_cost, qap_distances, num_ants, num_iterations, evaporation_rate):
    """
    Ant Colony Optimization (ACO) for the Quadratic Assignment Problem (QAP).

    Parameters:
    - qap_cost: 2D list (cost matrix), representing the flow between facilities.
    - qap_distances: 2D list (distance matrix), representing the distances between locations.
    - num_ants: int, the number of ants in the colony.
    - num_iterations: int, the number of iterations to perform.
    - evaporation_rate: float, the rate at which pheromones evaporate (0 < evaporation_rate < 1).

    Returns:
    - best_solution: List[int], the best solution (tour) found by the ants.
    - best_solution_cost: float, the cost of the best solution.
    """
    num_facilities = len(qap_cost)
    pheromone_matrix = np.ones((num_facilities, num_facilities))  # Initial pheromone levels
    best_solution = []
    best_solution_cost = float("inf")

    for _ in range(num_iterations):
        # Create the ants
        ants = [Ant(qap_cost, qap_distances, pheromone_matrix) for _ in range(num_ants)]

        # Each ant constructs a solution
        for ant in ants:
            start_facility = random.randint(0, num_facilities - 1)  # Random starting facility
            ant.construct_solution(start_facility)  # Construct solution

            # Update the best solution found
            if ant.cost < best_solution_cost:
                best_solution_cost = ant.cost
                best_solution = ant.tour  # Update the best solution

        # Update pheromones
        for ant in ants:
            for i in range(num_facilities - 1):
                facility_i = ant.tour[i]
                facility_j = ant.tour[i + 1]
                pheromone_matrix[facility_i][facility_j] += 1.0 / ant.cost
                pheromone_matrix[facility_j][facility_i] += 1.0 / ant.cost

            # Loop back to the starting facility
            facility_i = ant.tour[-1]
            facility_j = ant.tour[0]
            pheromone_matrix[facility_i][facility_j] += 1.0 / ant.cost
            pheromone_matrix[facility_j][facility_i] += 1.0 / ant.cost

        # Apply pheromone evaporation
        pheromone_matrix *= (1.0 - evaporation_rate)

    return best_solution, best_solution_cost

First, we prove it with the 9 cities problem, and we obtain the best possible result.

In [49]:
num_ants = 50
num_iterations = 1000
evaporation_rate = 0.1

best_route, best_tour_length = ant_colony_optimization(qap_cost_9, qap_distances_9, num_ants, num_iterations, evaporation_rate)

print("Best route found: ", best_route)
print("Best tour length: ", best_tour_length)

# Best solution founded: 94622
# Best solution: 94622

Best route found:  [8, 2, 5, 7, 6, 1, 0, 3, 4]
Best tour length:  94622


Now, we test it with different parameters.

In [62]:
num_ants = 50
num_iterations = 500
evaporation_rate = 0.1

best_route, best_tour_length = ant_colony_optimization(qap_cost_35, qap_distances_35, num_ants, num_iterations, evaporation_rate)

print("Best route found: ", best_route)
print("Best tour length: ", best_tour_length)

# Best solution founded: 2758798
# Best solution: 2422002

Best route found:  [17, 23, 16, 10, 13, 30, 21, 33, 9, 20, 19, 11, 24, 7, 29, 22, 15, 32, 31, 8, 28, 34, 26, 12, 27, 25, 1, 14, 6, 2, 4, 18, 5, 0, 3]
Best tour length:  2758798


## Evolution λ μ (Evolutionary Algorithm)

The **Evolution λ μ** algorithm is a population-based search strategy used to solve combinatorial optimization problems like the **Quadratic Assignment Problem (QAP)**. It is based on evolutionary principles and operates through a process of **selection**, **crossover (or recombination)**, **mutation**, and **replacement**. The algorithm is designed to improve a population of solutions over multiple generations, progressively refining the solutions towards better results.

In the Evolution λ μ approach:
- **λ (lambda)** represents the number of offspring (new solutions) generated in each generation.
- **μ (mu)** represents the number of parents selected to produce the offspring.

### Key Concepts in Evolution λ μ

1. **Selection**: A subset of the current population is selected as parents based on fitness, i.e., how good their solutions are. The selection process favors the selection of better solutions while maintaining some diversity.
  
2. **Crossover (Recombination)**: Two selected parents combine their features to create offspring. This is similar to how biological organisms inherit traits from both parents.
   
3. **Mutation**: Some offspring undergo random changes to introduce variability. This helps the algorithm escape local optima and explore a wider range of possible solutions.

4. **Replacement**: The newly generated offspring replace a portion of the current population, depending on the specific replacement strategy. The goal is to progressively improve the population over time.

### Evolution λ μ Algorithm

The basic idea is to start with a population of random solutions, evaluate them, and apply selection, crossover, and mutation to create a new generation of solutions. Over multiple generations, the population improves, ultimately converging to a high-quality solution for the QAP.

### Defining Variables for the Problem

Before implementing the algorithm, we first define the variables for the problem.

In [ ]:
mu = 5 # number of parents
lambd = 150  # number of offspring
num_generations = 5000 # number of generations

We have to define a mutation function, that receive a route and creverse the order of the nodes between indices i and k.

In [27]:
def mutation(route):
    """
    Perform a 2-opt swap on the given route by reversing the
    order of the nodes between indices i and k (inclusive).
    """
    route_list = list(route)
    i, k = np.random.randint(0, len(route), size = 2)
    if i > k:
        i, k = k, i
        
    new_route = route_list[:i] + route_list[i:k+1][::-1] + route_list[k+1:]
    return new_route

Here is the function that calculate the optimum, that only needs the two matrix and the previous defining of the variables mu, lambd and number of generations (num_generations).

In [ ]:
def calculate_optim(qap_cost, qap_distances):
    """
        Perform the Evolution λ μ algorithm to optimize the Quadratic Assignment Problem (QAP).

        Parameters:
        - qap_cost: 2D list (cost matrix), representing the flow between facilities.
        - qap_distances: 2D list (distance matrix), representing the distances between locations.

        Returns:
        - Best: List[int], the best solution found during the optimization process.
        - best_cost: float, the cost of the best solution.
    """
    Best = []
    P = np.array([randomSolution(qap_cost) for i in range(lambd)])

    for generation in range(num_generations):
        child_costs = []

        for child in P:
            child_cost = calculateCost(qap_cost, qap_distances, child)
            child_costs.append(child_cost)

            if len(Best) == 0 or child_cost < calculateCost(qap_cost, qap_distances, Best):
                Best = child

        # Order the costs and extract the indexes
        tuple_costs = [(value, index) for index, value in enumerate(child_costs)]
        tuple_costs_ordered = sorted(tuple_costs, key=lambda x: x[0])
        indexes = [index for value, index in tuple_costs_ordered]

        # Select the best mu individuals
        Q = P[indexes[:mu]]

        # Create the new generation
        P = np.empty((lambd, len(qap_cost)), dtype=int)
        aux = 0
        for i in range(mu):
            for j in range(lambd // mu):
                child = np.copy(Q[i])
                child = mutation(child)
                P[aux] = child
                aux += 1

    best_cost = calculateCost(qap_cost, qap_distances, Best)
    
    return Best, best_cost

We prove the algorithm with the 9 and 35 cities problem.

In [35]:
Best, best_cost = calculate_optim(qap_cost_9, qap_distances_9)
print("Best:", Best)
print("Best Final cost:", best_cost)

# Best solution founded: 96963
# Best solution: 94622

Best: [3 5 2 8 7 0 4 6 1]
Best Final cost: 109912


In [36]:
Best, best_cost = calculate_optim(qap_cost_35, qap_distances_35)
print("Best:", Best)
print("Best Final cost:", best_cost)

# Best solution founded: 2545412
# Best solution: 2422002

Best: [ 2  4 20 17  0 13 33 12 27 31 26  6 29  5 16  1 10 24 28 15  3 34 18 25
 14 11  9  8 22 32 21 30 19 23  7]
Best Final cost: 2545412


## Summary

- **First Improvement**:

    - **Advantages**:
        - Efficient in time, since it looks for and applies the first movement that improves the current solution.
        - You can quickly find acceptable solutions to problems with large search spaces.
        - Because of its time execution, we can reach a great solution using a loop.
        
    - **Disadvantages**:
        - You can get stuck at local optima without exploring the solution space in depth.
        - It does not guarantee finding the best overall solution.  
    

---

- **Best Improvement**:

    - **Advantages**:
        - Examines and selects the best available move in each iteration, which can lead to a more extensive search.
        - It has a better chance of finding better solutions than the First Improvement algorithm.
        - Because of its time execution, we can reach a great solution using a loop.
        
    - **Disadvantages**:
        - It can still suffer from stagnation at local optima.
    

---

- **Tabu Search**:

    - **Advantages**:
        - It avoids stagnation at local optima by using a tabu list that prohibits previous moves.
        - Promotes diversity of solutions by exploring different neighborhoods and movements.
        - You can find better quality solutions and get progressively better over time.
        
    - **Disadvantages**:
        - It requires a mechanism to manage and update the tabu list, which can add complexity to the implementation.
        - Requires longer execution time due to testing multiple neighborhoods, especially in the 35-cities case.
    

---

- **Ants (Ant Colony Optimization)**:

    - **Advantages**:
        - Utilizes the ability of ants to explore the solution space and find promising solutions.
        - Combines local and global information using pheromones to guide the search.
        - Can find high-quality solutions and adapt to changes in the environment.
        
    - **Disadvantages**:
        - It requires careful adjustment of parameters, such as the amount of pheromones and heuristic influence, to obtain good results.
        - May require a longer execution time due to the iterative construction of solutions by the ants.
    

---

- **Mu Lambda Evolution**:

    - **Advantages**:
        - Uses a population-based evolution strategy to find high-quality solutions.
        - Combines selection, crossing, and mutation to generate new solutions and progressively improve the quality of the population.
        - Can find high-quality or optimal solutions and adapt to different problems.
        
    - **Disadvantages**:
        - Requires proper adjustment of parameters, such as the size of the λ and μ groups, to obtain a balance between exploration and exploitation.
        - Requires longer run time due to handling and testing of entire populations, especially in the 35-cities case.
